In [1]:
%pip install seaborn
%pip install sklearn
%pip install xgboost imblearn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import tqdm

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import mutual_info_classif
from sklearn.neural_network import MLPClassifier
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
train_data = pd.read_csv("./train.csv")
df_train = pd.read_csv("./train.csv")
train_data.head()

In [ ]:
train_data["Survived"].value_counts()

In [ ]:
rdm = RandomUnderSampler()
X, Y = rdm.fit_resample(X, Y)

In [ ]:
train_data = pd.concat([X, Y], axis=1)

In [ ]:
test_data = pd.read_csv("./test.csv")
test_data.head()

In [ ]:
train_data["Survived"].value_counts()

In [ ]:
test = pd.read_csv("./test.csv")
train = pd.read_csv("./train.csv")
combine = [train, test]
print('Train columns:', train.columns.tolist())
print('Test columns:', test.columns.tolist())

In [ ]:
train.head()

In [ ]:
train.tail()

In [ ]:
train.info()
print('_'*40)
test.info()

In [ ]:
train.describe()

In [ ]:
train.describe(include=['O'])

In [ ]:
train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
train[['Sex', 'Survived']].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
train[['SibSp', 'Survived']].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
train[['Parch', 'Survived']].groupby(['Parch'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
g = sns.FacetGrid(train, col='Survived')
g.map(plt.hist, 'Age', bins=20)

In [ ]:
grid = sns.FacetGrid(train, col='Survived', row='Pclass', aspect=1.6)
grid.map(plt.hist, 'Age', alpha=.5, bins=20)
grid.add_legend();

In [ ]:
grid = sns.FacetGrid(train, row='Embarked', aspect=1.6)
grid.map(sns.pointplot, 'Pclass', 'Survived', 'Sex', palette='deep')
grid.add_legend()

In [ ]:
grid = sns.FacetGrid(train, row='Embarked', col='Survived', aspect=1.6)
grid.map(sns.barplot, 'Sex', 'Fare', alpha=.5, ci=None)
grid.add_legend()

In [ ]:
print("Before", train.shape, test.shape, combine[0].shape, combine[1].shape)
train = train.drop(['Ticket', 'Cabin'], axis = 1)
test = test.drop(['Ticket', 'Cabin'], axis=1)
combine = [train, test]

"After", train.shape, test.shape, combine[0].shape, combine[1].shape

In [ ]:
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

pd.crosstab(train['Title'], train['Sex'])

In [ ]:
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    
train[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

In [ ]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

train.head()

In [ ]:
train = train.drop(['Name', 'PassengerId'], axis=1)
test = test.drop(['Name'], axis=1)
combine = [train, test]
train.shape, test.shape

In [ ]:
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map({'female': 1,'male': 0 }).astype(int)

train.head()

In [ ]:
grid = sns.FacetGrid(train, row='Pclass', col='Sex', aspect=1.6)
grid.map(plt.hist, 'Age', alpha=.5, bins=20)
grid.add_legend()

In [ ]:
guess_ages = np.zeros((2,3))
guess_ages

In [ ]:
for dataset in combine:
    for i in range(0, 2):
        for j in range(0, 3):
            guess_df = dataset[(dataset['Sex'] == i) & (dataset['Pclass'] == j+1)]['Age'].dropna()
            age_guess = guess_df.median()
            guess_ages[i, j] = int(age_guess/0.5 + 0.5) * 0.5
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[(dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1), 'Age'] = guess_ages[i, j]
    dataset['Age'] = dataset['Age'].astype(int)
train.head()

In [ ]:
train['AgeBand'] = pd.cut(train['Age'], 5)
train[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

In [ ]:
for dataset in combine:
    dataset.loc[dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[dataset['Age'] > 64,'Age']
train.head()

In [ ]:
train = train.drop(['AgeBand'], axis = 1)
combine = [train, test]
train.head()

In [ ]:
for dataset in combine:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
train[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
for dataset in combine:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

train[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean()

In [ ]:
train.head()

In [ ]:
guess_fare = np.zeros((2,3))
guess_fare

In [ ]:
for dataset in combine:
    for i in range(0, 2):
        for j in range(0, 3):
            guess = dataset[(dataset['Sex'] == i) & (dataset['Pclass'] == j+1)]['Fare'].dropna()
            fare_guess = guess.median()
            guess_fare[i, j] = int(fare_guess/0.5 + 0.5) * 0.5
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[(dataset.Fare.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1), 'Fare'] = guess_fare[i, j]
    dataset['Fare'] = dataset['Fare'].astype(int)
train.head()

In [ ]:
train.tail()

In [ ]:
train['FareBand'] = pd.cut(train['Fare'], 5)
train[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)

In [ ]:
for dataset in combine:
    dataset.loc[dataset['Fare'] <= 102.4, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 102.4) & (dataset['Fare'] <= 204.8), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 204.8) & (dataset['Fare'] <= 307.2), 'Fare'] = 2
    dataset.loc[(dataset['Fare'] > 307.2) & (dataset['Fare'] <= 409.6), 'Fare'] = 3
    dataset.loc[dataset['Fare'] > 409.6,'Fare']
train.head()

In [ ]:
train.tail()

In [ ]:
train

In [ ]:
train = train.drop(['FareBand'], axis = 1)
combine = [train, test]
train

In [ ]:
guess_embarked = np.zeros((2,3))
guess_embarked

In [ ]:
for dataset in combine:
    dataset.loc[dataset['Embarked'] == 'S', 'Embarked'] = 0
    dataset.loc[dataset['Embarked'] == 'C', 'Embarked'] = 1
    dataset.loc[dataset['Embarked'] == 'Q', 'Embarked'] = 2
train.head()

In [ ]:
for dataset in combine:
    dataset.loc[dataset['Pclass'] == None, 'Pclass'] = 3
    dataset.loc[dataset['Sex'] == None, 'Sex'] = rnd.randint(0, 1)
    dataset.loc[dataset['Age'] == None, 'Age'] = rnd.randint(0, 3)

In [ ]:
train.info()

In [ ]:
train['Embarked'] = train['Embarked'].fillna(0)

In [ ]:
train['Embarked'].unique()

In [ ]:
X = train.drop(columns=['Survived'])
Y = train['Survived']

In [ ]:
result = []
for i in range(0, 100):
    j = rnd.randint(1, 100)
    k = rnd.randint(1, 100)
    model_rf = RandomForestClassifier(random_state=i, max_depth = j, n_estimators = k)
    result.append([cross_val_score(model_rf, X, Y, cv=5), j, k])

[0.73743017, 0.81460674, 0.80898876, 0.80898876, 0.85955056]), random_state=97, max_depth=5, n_estimators=3]

In [ ]:
result = []
for i in range(0, 100):
    j = rnd.randint(1, 100)
    k = rnd.randint(1, 100)
    model_rf = DecisionTreeClassifier(random_state= i, splitter = "best", max_depth = j)
    result.append([cross_val_score(model_rf, X, Y, cv=5), j, k])

[0.81564246, 0.8258427 , 0.8258427 , 0.79213483, 0.85393258]), random_state=64, max_depth=4]

result = []
for i in range(0, 100):
    j = rnd.randint(1, 100)
    k = rnd.randint(1, 100)
    model_rf = MLPClassifier(random_state=i, hidden_layer_sizes=950, max_iter=1000)
    result.append([cross_val_score(model_rf, X, Y, cv=5), j, k])

[0.81005587, 0.83707865, 0.84269663, 0.80898876, 0.88764045]) random_state=100

In [ ]:
result

In [ ]:
test.head()

In [ ]:
X = train_data.drop(columns=["Survived"], axis=1)
Y = train_data["Survived"]
rdm = RandomUnderSampler()
X, Y = rdm.fit_resample(X, Y)

In [ ]:
X_train = train.drop(columns=["Survived"], axis=1)
Y_train = train["Survived"]
X_test = test.drop(columns=["PassengerId"], axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
logreg.score(X_train, Y_train)

In [ ]:
coeff = pd.DataFrame(train.columns.delete(0))
coeff.columns = ['Feature']
coeff["Correlation"] = pd.Series(logreg.coef_[0])

coeff.sort_values(by='Correlation', ascending=False)

In [ ]:
svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
svc.score(X_train, Y_train)

In [ ]:
for i in range(1, 100, 2):
    knn = KNeighborsClassifier(n_neighbors = i)
    knn.fit(X_train, Y_train)
    Y_pred = knn.predict(X_test)
    knn.score(X_train, Y_train)
    result.append([knn.score(X_train, Y_train), i])
result

In [ ]:
gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
Y_pred = gaussian.predict(X_test)
gaussian.score(X_train, Y_train)

In [ ]:
perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
Y_pred = perceptron.predict(X_test)
perceptron.score(X_train, Y_train)

In [ ]:
linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
Y_pred = linear_svc.predict(X_test)
linear_svc.score(X_train, Y_train)

In [ ]:
sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
Y_pred = linear_svc.predict(X_test)
sgd.score(X_train, Y_train)

In [ ]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
decision_tree.score(X_train, Y_train)

In [ ]:
random_forest = RandomForestClassifier(n_estimators = 100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)X_train

In [ ]:
mlp = MLPClassifier()
mlp.fit(X_train, Y_train)
Y_pred = mlp.predict(X_test)
mlp.score(X_train, Y_train)

j = 1
k = 1
result = []
for i in range(1, 100):
    random_forest = RandomForestClassifier(n_estimators = i, max_depth=j, random_state=k)
    random_forest.fit(X_train, Y_train)
    Y_pred = random_forest.predict(X_test)
    random_forest.score(X_train, Y_train)
    for j in range(1, 100):
        random_forest = RandomForestClassifier(n_estimators = i, max_depth=j, random_state=k)
        random_forest.fit(X_train, Y_train)
        Y_pred = random_forest.predict(X_test)
        random_forest.score(X_train, Y_train)
        for  k in range(1, 100):
            random_forest = RandomForestClassifier(n_estimators = i, max_depth=j, random_state=k)
            random_forest.fit(X_train, Y_train)
            Y_pred = random_forest.predict(X_test)
            random_forest.score(X_train, Y_train)
    result = ([random_forest.score(X_train, Y_train), j, k])

In [ ]:
acc_svc = svc.score(X_train, Y_train)
acc_knn = knn.score(X_train, Y_train)
acc_log = logreg.score(X_train, Y_train)
acc_rf = random_forest.score(X_train, Y_train)
acc_gaussian = gaussian.score(X_train, Y_train)
acc_perceptron = perceptron.score(X_train, Y_train)
acc_sgd = sgd.score(X_train, Y_train)
acc_linear = linear_svc.score(X_train, Y_train)
acc_dt = decision_tree.score(X_train, Y_train)
acc_mlp = mlp.score(X_train, Y_train)

In [ ]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 'Random Forest', 'Naive Bayes', 'Perceptron', 'Stochastic Gradient Decent', 'Linear SVC', 'Decision Tree', 'MLP'],
    'Score': [acc_svc, acc_knn, acc_log, acc_rf, acc_gaussian, acc_perceptron, acc_sgd, acc_linear, acc_dt, acc_mlp]})
models.sort_values(by='Score', ascending=False)

In [ ]:
submission = pd.DataFrame({"PassengerId": test["PassengerId"], "Survived": Y_pred})

In [ ]:
submission.to_csv('./submission.csv', index=False)

In [ ]:
dataset.to_csv("./New_Dataset", index=False)